# 1. importation des librairies et preparation des données

## 1.1 importation des librairies

In [4332]:

import pandas as pd

In [4333]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

## 1.2 importation des dataframes

In [4334]:
aid = pd.read_csv('../DA-P4/sources/aide_alimentaire.csv')
dispo = pd.read_csv('../DA-P4/sources/dispo_alimentaire.csv')
pop = pd.read_csv('../DA-P4/sources/population.csv')
lack = pd.read_csv('../DA-P4/sources/sous_nutrition.csv')
fao_stat =  pd.read_csv('../DA-P4/sources/FAOSTAT.csv')

## 1.3 preparation des dataframes

In [4335]:
# preparation du fichier avec les données de sous nutritions manquantes

fao_stat= fao_stat.drop(['Domaine', 'Code Domaine','Code zone (M49)', 'Code Élément','Élément','Code Produit', 'Code année', 'Note' ], axis=1)
fao_stat.isna()
fao_stat= fao_stat.dropna(axis=0)
fao_stat['Valeur'] = fao_stat['Valeur'].astype(float)
fao_stat = fao_stat[fao_stat['Année']=='2016-2018']

In [4336]:
# preparation du fichier population

pop = pop[pop['Année']==2017]
pop['Valeur'] = pop['Valeur'].astype(float)

In [4337]:
# preparation du fichier sous nutrition

new_lack = lack.copy()
new_lack = new_lack[new_lack['Année']=='2016-2018']
new_lack['Valeur'] = new_lack['Valeur'].astype(float)

In [4338]:
# fusion des fichiers fao_stats et population, et conversion du nombre d'habitants de pourcentage a nombre reel, exprimé en milliers

new = pd.merge(fao_stat, pop, on=["Zone"], how='left')
new["Valeur1"]= ((new.Valeur_y * new.Valeur_x / 100) * 0.001)
new["Valeur1"] = new["Valeur1"].round(decimals=1)
new = new.drop(["Produit", "Année_x", "Unité", "Symbole"], axis=1)

In [4339]:
# fusion entre le fichier des données de sous nutrions, et du nouveau fichier possedant les valeurs manquantes

Lack_filled = pd.merge(new_lack, new, on='Zone', how="left")
Lack_filled['Valeur'].fillna(Lack_filled['Valeur1'],inplace=True)
Lack_filled = Lack_filled.drop(['Année_y', "Valeur1", 'Valeur_x', 'Valeur_y'], axis=1)


In [4340]:
# fusion du nouveau fichier de données avec les données de population
Lack_filled = pd.merge(Lack_filled, pop, on='Zone', how='left')
Lack_filled = Lack_filled.drop(['Description du Symbole'], axis=1)
Lack_filled = Lack_filled.dropna(axis=0)

# 2. Annalyse generale

## 2.1 proportion de personnes en état de sous-nutrition 

In [4341]:
# addition des données de population et des données de sous-nutrition, pour les pays renseignés, puis calcul de pourcentage
world_pop = Lack_filled['Valeur_y'].sum()
world_lack = Lack_filled['Valeur_x'].sum()
print(f'Le taux de personnes en etat de sous-nutrition est de {((world_lack/world_pop) * 100000).round(decimals=1)}%')


Le taux de personnes en etat de sous-nutrition est de 9.1%


## 2.2 nombre théorique de personnes qui pourraient être nourries

In [4342]:
#addition du nombre de kcal/personne/jour disponible dans chaque pays
new_dispo = pd.read_csv('../DA-P4/sources/dispo_alimentaire.csv')
new_dispo = new_dispo.groupby('Zone').sum(['Disponibilité alimentaire (Kcal/personne/jour)'])


In [4343]:
#ajout des données de population par pays pour l'année 2017, puis multiplication de la donnée Kcal/personne/jour par la donnée de population
#pour obtenir une donnée de Kcal/pays/jour
new_dispo = pd.merge(new_dispo, pop, on=['Zone'], how="left")
new_dispo.drop(new_dispo.loc[new_dispo['Année']!=2017.0].index, inplace=True)
new_dispo["Kcal/pays/jour"]= (new_dispo['Disponibilité alimentaire (Kcal/personne/jour)'] * new_dispo["Valeur"])

In [4344]:
# addition des Kcal/pays/jour pour obtenir un nombre de Kcal/jour au niveau mondial, puis addition des populations de chaque pays.
# un humain consomme en moyenne 2500 calories par jour, ce qui permet de calculer le nombre d'humains que l'on pourrait théoriquement nourrir.
#calcul du pourcentage d'humains que l'on pourrait nourrir
daily_cal = new_dispo["Kcal/pays/jour"].sum()
world_pop = new_dispo['Valeur'].sum()
fed_humans = daily_cal / 2500
print(f"Le taux d'humains théorique pouvant etre nourri est de {round((fed_humans / world_pop) * 100)} % de la population mondiale")

Le taux d'humains théorique pouvant etre nourri est de 115 % de la population mondiale


## 2.3 nombre théorique de personnes qui pourraient être nourries a partir de la disponibilité alimentaire des produits végétaux 

In [4345]:
#selection de la disponibilité alimentaire des produits vegetaux, puis addition par pays
vegetal_dispo = pd.read_csv('../DA-P4/sources/dispo_alimentaire.csv')
vegetal_dispo = vegetal_dispo.loc[vegetal_dispo['Origine']== 'vegetale']
vegetal_dispo = vegetal_dispo.groupby('Zone').sum(['Disponibilité alimentaire (Kcal/personne/jour)'])

In [4346]:
#ajout des données de population et selection de la bonne année
vegetal_dispo = pd.merge(vegetal_dispo, pop, on=['Zone'], how="left")
vegetal_dispo.drop(vegetal_dispo.loc[vegetal_dispo['Année']!=2017.0].index, inplace=True)

In [4347]:
# meme calculs que pour lq question precedente, calcul du nombre d'humains 
# pouvant théoriquement être nourris uniquement grace aux produits vegetaux
vegetal_dispo["Kcal/pays/jour"]= (vegetal_dispo['Disponibilité alimentaire (Kcal/personne/jour)']* vegetal_dispo["Valeur"])
daily_cal_veg = vegetal_dispo["Kcal/pays/jour"].sum()
fed_humans_veg = daily_cal_veg / 2500
print(f"Le taux d'humains théorique pouvant etre nourri est de {round((fed_humans_veg / world_pop) * 100)}% de la population mondiale")

Le taux d'humains théorique pouvant etre nourri est de 95% de la population mondiale


## 2.3 utilisation de la disponibilité intérieure

In [4348]:
# calcul de la part des aliments pour animaux, de la nourriture, et des pertes dans la disponibilité intérieure
dispo_sum = dispo['Disponibilité intérieure'].sum()
animals_sum = dispo['Aliments pour animaux'].sum()
lost_sum = dispo['Pertes'].sum()
nut_sum = dispo['Nourriture'].sum()
print(f"La part des aliments pour animaux est de {(animals_sum * 100 / dispo_sum).round(decimals=1)}% de la disponibilité intérieure")
print(f"La part des pertes est de {(lost_sum * 100 / dispo_sum).round(decimals=1)}% de la disponibilité intérieure")
print(f"La nourriture correspond à {(nut_sum * 100 / dispo_sum).round(decimals=1)}% de la disponibilité intérieure")


La part des aliments pour animaux est de 13.2% de la disponibilité intérieure
La part des pertes est de 4.6% de la disponibilité intérieure
La nourriture correspond à 49.5% de la disponibilité intérieure


# 3. Questions specifiques

## 3.1  lien logique entre l’ensemble des 10 variables de l'utilisation de la disponibilité intérieure

disponibilité interieure = 
importation + production - exportations + variation de stock = 
nourriture + pertes + semences + aliments pour animaux + autre utilisations + traitement

## 3.2 utilisation des céréales

In [4349]:
# calcul des données de nutrition et aliments pour animaux, uniquement pour les cereales,
# puis calcul de leur proportion par rapport a la disponibilité intérieure des céréqles
cereal = ['Avoine','Blé','Céréales, Autres','Maïs','Miel','Millet',
'Orge','Riz (Eq Blanchi)','Seigle','Sorgho', "Alpiste","Triticale","Céréales élangées",
"Fonio","Quinoa","Riz, paddy","Sarrasin, blé noir"]
dispo_cereal = dispo[dispo['Produit'].isin(cereal)]
animals = dispo_cereal['Aliments pour animaux'].sum()
nut = dispo_cereal['Nourriture'].sum()
disp = dispo_cereal['Disponibilité intérieure'].sum()
nut = (nut / disp) * 100
animals = (animals / disp) * 100
print(f"La part de nourriture de la disponibilité intérieure des céréales est de {nut.round(decimals=1)}%")
print(f"La part des aliments pour animaux de la disponibilité intérieure des céréales est de {animals.round(decimals=1)}%")




La part de nourriture de la disponibilité intérieure des céréales est de 42.8%
La part des aliments pour animaux de la disponibilité intérieure des céréales est de 36.3%


## 3.3 utilisation du manioc par la Thaïlande

In [4350]:
dispo_thai = dispo[dispo['Zone']=='Thaïlande']
dispo_thai = dispo_thai[dispo_thai['Produit']=='Manioc']
export = dispo_thai['Exportations - Quantité'].sum()
prod = dispo_thai['Production'].sum()
prop = (export/prod) * 100
under_nut_thai = lack[lack['Zone']=='Thaïlande']
under_nut_thai = under_nut_thai[under_nut_thai['Année']=='2016-2018']
print(f"La thailande produit {prod.astype(int)} milliers de tonnes de manioc")
print(f"La thailande exporte {export.astype(int)} milliers de tonnes de manioc")
print(f"La thailande exporte {prop.round(decimals=0).astype(int)}% de son manioc")
print(f"La thailande possède {under_nut_thai['Valeur'].sum()} millions de personnes en état de sous-nutrition")

# exportation de plus de la moitie de produciton de manioc en thailande, avec un nombre de personnes en sous nutrition supérieur a 6 millions

La thailande produit 30228 milliers de tonnes de manioc
La thailande exporte 25214 milliers de tonnes de manioc
La thailande exporte 83% de son manioc
La thailande possède 6.2 millions de personnes en état de sous-nutrition


# 4. annalyse approfondie

## 4.1 pays dont la proportion de personnes sous-alimentées est la plus elevée en 2017

In [4351]:
# Division des données de population par 1000 pour avoir la meme unité que les personnes sous alimentées (exprimées en millions)
# Calcul du pourcentage, puis affichage par ordre decroissant de proportion
sortd = Lack_filled
sortd.rename(columns={'Valeur_x':'personnes sous alimentées', 'Valeur_y':'population'}, inplace=True)
#sortd.rename(columns={'Valeur_x':'personnes sous alimentées', 'Valeur_y':'population'}, inplace=True)
sortd['population'] = sortd['population'] / 1000
sortd['prop'] = (sortd['personnes sous alimentées'] / sortd['population']) * 100
sortd = sortd.sort_values(by = 'prop', ascending = False)
display(sortd[:10])
# à noter que les pays avec un nombre faible d'habitants n'apparaissent pas dans ce classement

,Zone,Année_x,personnes sous alimentées,Année_y,population,prop
78,Haïti,2016-2018,5.3,2017,10.982366,48.259182
157,République populaire démocratique de Corée,2016-2018,12.0,2017,25.429825,47.188685
200,Yémen,2016-2018,13.0,2017,27.834819,46.704094
151,République centrafricaine,2016-2018,2.1,2017,4.596023,45.691677
108,Madagascar,2016-2018,10.5,2017,25.570512,41.062924
103,Libéria,2016-2018,1.8,2017,4.702226,38.279742
100,Lesotho,2016-2018,0.8,2017,2.091534,38.249438
183,Tchad,2016-2018,5.7,2017,15.016753,37.957606
154,République démocratique du Congo,2016-2018,29.6,2017,81.398764,36.364189
161,Rwanda,2016-2018,4.2,2017,11.980961,35.055619


## 4.2 pays ayant le plus bénéficié d’aide depuis 2013

In [4352]:
# addition de l'aide recue chaque année depuis 2013, par pays, puis affichage par ordre decroissant
aid = aid.groupby("Pays bénéficiaire").sum()
aid = aid.sort_values(by = 'Valeur', ascending = False)
aid = aid.drop(['Année'], axis=1)
aid.head(10)

,Valeur
Pays bénéficiaire,
République arabe syrienne,1858943
Éthiopie,1381294
Yémen,1206484
Soudan du Sud,695248
Soudan,669784
Kenya,552836
Bangladesh,348188
Somalie,292678
République démocratique du Congo,288502


## 4.3 pays ayant le plus de disponibilité/habitant

In [4353]:
dispo = dispo.groupby("Zone").sum(['Disponibilité alimentaire (Kcal/personne/jour)'])
dispo = dispo.sort_values(by = 'Disponibilité alimentaire (Kcal/personne/jour)', ascending = False)
display(dispo['Disponibilité alimentaire (Kcal/personne/jour)'][:10])

Zone
Autriche                 3770.0
Belgique                 3737.0
Turquie                  3708.0
États-Unis d'Amérique    3682.0
Israël                   3610.0
Irlande                  3602.0
Italie                   3578.0
Luxembourg               3540.0
Égypte                   3518.0
Allemagne                3503.0
Name: Disponibilité alimentaire (Kcal/personne/jour), dtype: float64

## 4.3 pays ayant le moins de disponibilité/habitant

In [4354]:
dispo = dispo.sort_values(by = 'Disponibilité alimentaire (Kcal/personne/jour)', ascending = True)
display(dispo['Disponibilité alimentaire (Kcal/personne/jour)'][:10])

Zone
République centrafricaine                     1879.0
Zambie                                        1924.0
Madagascar                                    2056.0
Afghanistan                                   2087.0
Haïti                                         2089.0
République populaire démocratique de Corée    2093.0
Tchad                                         2109.0
Zimbabwe                                      2113.0
Ouganda                                       2126.0
Éthiopie                                      2129.0
Name: Disponibilité alimentaire (Kcal/personne/jour), dtype: float64

# 5. conclusion

Il existe suffisament de nourriture pour nourrir l'intégralité de la population mondiale - 115%
En utilisant uniquement les produits végétaux, ce n'est plus le cas - 95%

Les céréales sont utilisées presque autant pour les animaux que pour les humains.

La thailande exporte l'essentiel de sa production de manioc.

Les pays ayant reçu le plus d'aide alimentaire ne sont pas ceux qui possedent la disponibilité alimentaire la plus faible,
exception faite de l'ethiopie.